# FTX reader

In [31]:
import matplotlib
%matplotlib inline
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import plotly.express as px

import sys
sys.path.append('../')
from history_script import *

import os
directory='C:/Users/david/Dropbox/mobilier/crypto/DONOTDELETE_temporary_parquets/'
end_time=datetime.today().replace(minute=0,second=0,microsecond=0)
start_time=end_time-timedelta(weeks=1)
modes=[('h','high'),('c','close'),('l','low')]
coins=['OMG','BOBA']

In [2]:
futures_df=ftx_history(dirname='',timeframe='1h',coin_list=coins)

{'apiKey': True, 'secret': True, 'uid': False, 'login': False, 'password': False, 'twofa': False, 'privateKey': False, 'walletAddress': False, 'token': False}


C:\Users\david\Puffin\DerivativeArbitrage\JupyterNotebooks\..\history_script.py:31: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  enriched = enricher(exchange, futures, holding_period, equity=equity,


funding_history: OMG-PERP
rate_history: OMG-PERP


C:\Users\david\Puffin\DerivativeArbitrage\JupyterNotebooks\..\ftx_history.py:26: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  future_rate_data=pd.concat([rate_history(f, exchange, end, start, timeframe,dirname)


rate_history: OMG-1231
price_history: OMG-PERP
price_history: OMG-1231
price_history: OMG/USD
borrow_history: OMG
borrow_history: USD


In [35]:
prices=futures_df[[c for c in futures_df.columns for string in ['mark/c','price/c'] if string in c]]
rates=futures_df[[c for c in futures_df.columns for string in ['rate/c','rate/borrow'] if string in c]]
prices.join(rates).iplot(secondary_y=list(rates.columns))

## stopout: show min/max of premiums

In [ ]:
def compute_premium(df: pd.DataFrame,mode=1) -> pd.Series:
    return (df.filter(like='/mark/'+ modes[mode][0]).iloc[:,0]/df.filter(like='/indexes/'+modes[mode][1]).iloc[:,0]-1)*10000
def compile_list(premium_list: list) -> pd.DataFrame:
    premium=premium_list[0]
    for df in premium_list[1:]:
        premium=premium.join(df,how='outer')
    return premium[~premium.index.duplicated()]

In [ ]:
futures_df['premium']=compute_premium(futures_df)

In [ ]:
premium=compute_premium(futures_df,mode=0)
ewMax=premium.resample('1h').max()
ewMax=ewMax[~ewMax.index.duplicated()]
fig = px.line(ewMax,title="fut high - spot high")#, x='date', y="GOOG")
fig.show()

In [ ]:
premium=compute_premium(futures_df,mode=2)
ewMin=premium.resample('1h').min()
ewMin=ewMin[~ewMin.index.duplicated()]
fig = px.line(ewMin,title="fut low - spot low")#, x='date', y="GOOG")
fig.show()

## spot vs funding